In [7]:
import numpy as np
import scipy.special

In [8]:
class AutoEncoder:

	def __init__(self, spec:dict, learning_rate:int=0.05):
		if not len(spec["hidden_layers"]):
			raise ValueError("Models must contain at least one hidden layer.")
		if not spec["input_layer"]["nodes"] or not spec["output_layer"]["nodes"] or 0 in [layer["nodes"] for layer in spec["hidden_layers"]]:
			raise ValueError("All layers must contain at least one node.")

		self.spec = spec
		self.spec["input_layer"]["weights"] = np.random.normal(
			loc=0.0, 
			scale=pow(self.spec["input_layer"]["nodes"], -0.5), 
			size=(self.spec["hidden_layers"][0]["nodes"], self.spec["input_layer"]["nodes"])
		)
		for i, hidden_layer in enumerate(spec["hidden_layers"]):
			if i < len(spec["hidden_layers"]) - 1:
				next_layer = spec["hidden_layers"][i+1]
			else:
				next_layer = spec["output_layer"]
			self.spec["hidden_layers"][i]["weights"] = np.random.normal(
				loc=0.0, 
				scale=pow(hidden_layer["nodes"], -0.5), 
				size=(next_layer["nodes"], hidden_layer["nodes"])
			)
		self.model_summary()
	
	def train(self, instances, epochs=1):
		pass
	
	def query(self, instances):
		pass
	
	def evaluate(self, instances):
		pass

	def model_summary(self):
		print("\n".join([
			"========================================",
			"Initialised model",
			"========================================",
			"Input layer: 		{} nodes".format(self.spec["input_layer"]["nodes"]),
			*["Hidden layer ({}):	{} nodes".format(i, layer["nodes"]) for i, layer in enumerate(self.spec["hidden_layers"])],
			"Output layer:		{} nodes".format(self.spec["output_layer"]["nodes"]),
			"========================================",
		]))

In [9]:
spec = {
	"input_layer": { "nodes": 1 },
	"hidden_layers": [
		{ "nodes": 2, "activation_func": lambda x: scipy.special.expit(x) },
		{ "nodes": 1, "activation_func": lambda x: scipy.special.expit(x) },
		{ "nodes": 2, "activation_func": lambda x: scipy.special.expit(x) },
	],
	"output_layer": { 
		"nodes": 1,"activation_func": lambda x: scipy.special.expit(x) 
	},
}

model = AutoEncoder(spec, 0.05)

Initialised model
Input layer: 		1 nodes
Hidden layer (0):	2 nodes
Hidden layer (1):	1 nodes
Hidden layer (2):	2 nodes
Output layer:		1 nodes
